In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

# Rotation curves

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import astropy.units as u
from astropy.constants import G

import snapshot_obj 
import subhalo
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(subhalo)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Get data

In [ ]:
snap_id = 127
data = {"spec-p082" : 
        {"M31" : {"halo_id" : (1,0),
                  "color" : "red",
                  "linestyle" : "solid"},
         "MW" : {"halo_id" : (1,1),
                 "color" : "red",
                 "linestyle" : "dashed"}},
        "spec-p082-LR" :
        {"M31" : {"halo_id" : (1,0),
                  "color" : "pink",
                  "linestyle" : "solid"},
         "MW" : {"halo_id" : (1,1),
                  "color" : "pink",
                  "linestyle" : "dashed"}},
        "spec-p084-LR" :
        {"Spurious" : {"halo_id" : (1,0),
                       "color" : "lightblue",
                       "linestyle" : "dotted"}},
        "plain-LCDM" : 
        {"M31" : {"halo_id" : (1,0),
                  "color" : "black",
                  "linestyle" : "solid"},
         "MW" : {"halo_id" : (2,0),
                 "color" : "black",
                 "linestyle" : "dashed"}},
        "plain-LCDM-LR" :
        {"M31" : {"halo_id" : (1,0),
                  "color" : "gray",
                  "linestyle" : "solid"},
         "MW" : {"halo_id" : (2,0),
                 "color" : "gray",
                 "linestyle" : "dashed"}}
       }

sim_ids = [ "V1_MR_curvaton_p082_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix",
           "V1_MR_fix", "V1_LR_fix"]

In [ ]:
# Set plot limits:
x_down = 0; x_up = 60
y_down = 0; y_up = 300

In [ ]:
for sim_id, sim_name in zip(sim_ids, data.keys()):
    snap = snapshot_obj.Snapshot(sim_id, snap_id, name=sim_name)
    
    rot_curves = snap.get_subhalos('Vcirc', 
                                   group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos('SubOffset', 
                                   group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    
    max_point = snap.get_subhalos("Max_Vcirc", group="Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    v1kpc = snap.get_subhalos("V1kpc", group="Extended") * units.cm.to(units.km)
    
    # Iterate over central galaxies and save rotation curves to data dictionary:
    for galaxy in data[sim_name].keys():
        central_id = data[sim_name][galaxy]["halo_id"]
        idx = snap.index_of_halo(central_id[0], central_id[1])
        data[sim_name][galaxy]["Vmax"] = vmax[idx]
        data[sim_name][galaxy]["Rmax"] = rmax[idx]
        data[sim_name][galaxy]["Vcirc"] = v_circ[sub_offset[idx]:sub_offset[idx+1]]
        data[sim_name][galaxy]["Radius"] = radii[sub_offset[idx]:sub_offset[idx+1]]
        data[sim_name][galaxy]["V1kpc"] = v1kpc[idx]

In [ ]:
# Construct saving location:
filename = 'rotation_curves_of_centrals.png'     
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(figsize=(6,4))

# Set axes:
axes.set_xlim(x_down, x_up)
axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$')
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

# Galaxy text:
#axes.set_title('Rotation curve')

for sim_name, sim_data in data.items():
    for galaxy, gal_data in sim_data.items():
        axes.plot(gal_data["Radius"], gal_data["Vcirc"], 
                  c=gal_data["color"], linestyle=gal_data["linestyle"],
                  label="{}: {}".format(sim_name, gal_data["halo_id"])
                 )

        #axes.axvline(entry["Rmax"], c=color[i], linestyle='dashed')
        #axes.axhline(entry["V1kpc"], c=color[i], linestyle='dotted')
    
plt.legend(ncol=2, loc='lower right')
plt.tight_layout()

plt.savefig(os.path.join(path,filename), dpi=300)